## Disciplina:  Algoritmo II

## Trabalho Prático 01 -- Aplicações de algoritmos geométricos

### Aluno:       Christian Vieira 
### Matrícula:   2014106325

## Objetivos

O objetivo deste trabalho prático consiste na abordagem de aspectos práticos de implementação dos algoritmos geométricos lecionados durante as aulas teóricas da disciplina de Algoritmos II. Particularmente, aspectos práticos das árvores k-dimensionais serão abordados com o intuito de exibir possíveis aplicações com o intuito
de solucionar problemas em outras áreas da computação. Além dos detalhes da implementação da estrutura de dados em si, o trabalho prático propicia aplicação em outros contextos não exatamente focados em geometria computacional, mas tendo por base um algoritmo e estrutura de dados estudados no contexto de geometria computacional.


In [1]:
import sys
import os
import numpy as np
import pandas as pd

# Definicoes bases
BASE_DE_DADOS            = "base-de-dados/"
EXTENSAO_DOS_ARQUIVOS    = ".csv"
SEMENTE_DE_ALEATORIDADE  = 21
NUM_DE_VIZINHOS          = 9
PORCAO_DE_TESTES         = 0.7

### KdTree
A classe abaixo implementa uma KdTree conforme algoritmo visto em sala de aula.

In [2]:
class KdTree:
    def __init__(self, k):
        self.k         = k
        self.dado      = None
        self.direcao   = None
        self.esquerda  = None
        self.direita   = None

    def __calcula_mediana(self, arr):
        l = np.sort(arr)
        l = np.unique(l)
        n = len(l) 

        if  n % 2:
            i = int((n - 1)/2)
        else:
            i = int(n/2 - 1)
        return l[i]
    
    def constroiKdTree(self, pontos, profundidade, direcao):
        k = self.k
        if len(pontos) == 1:
            self.dado = pontos[0]
            self.direcao = direcao
        else:
            l =  self.__calcula_mediana(pontos.T[profundidade])
            d  = profundidade
            pontos_1 = pontos[pontos.T[profundidade] <= l]
            pontos_2 = pontos[pontos.T[profundidade] > l]
            for i in range(k):
                if len(pontos_1) == 0 or len(pontos_2) == 0:
                    d = (d + 1) % k
                    l =  self.__calcula_mediana(pontos.T[d])
                    pontos_1 = pontos[pontos.T[d] <= l]
                    pontos_2 = pontos[pontos.T[d] > l]
                else:
                    break
            
            n1 = KdTree(k)
            n2 = KdTree(k)

            n1.constroiKdTree(pontos_1, (d + 1) % k, 0)
            n2.constroiKdTree(pontos_2, (d + 1) % k, 1)
            
            self.esquerda  = n1
            self.direita = n2
            self.dado  = [l, d]
            self.direcao = direcao


    def constroi(self, pontos):
        self.constroiKdTree(pontos, 0, 0)

    
    def __percorre_rec(self, arr):
        if (self.direita is None and self.esquerda is None):
            arr.append(self.dado)
        else:
            if self.direita:
                self.direita.__percorre_rec(arr)
            if self.esquerda:
                self.esquerda.__percorre_rec(arr)


    def percorre(self):
        arr = []
        self.__percorre_rec(arr)
        return arr

    def __intersecao_das_regioes(self, regiao_1, arr, direcao):
        # calcula a intersecao entre uma regiao r1 e a regiao definida por uma divisao no 
        # hiperespaco na dimensao arr[1], definida por arr[0] e direcao
        profundidade = arr[1]
        regiao_3     = regiao_1.copy()
        esq          = arr[0]
        if direcao == 0:
            r_esquerda = [np.NINF, esq]
            
            # Verifica se dois intervalos tem intersecao nao nula
            r1 = regiao_1[profundidade]
            r2 = r_esquerda
            min1, max1 = r1[0], r1[1]
            min2, max2 = r2[0], r2[1]
            flag = False if (max2 <= min1 or min2 >= max1) else True
            #if interval_intersect(regiao_1[profundidade], r_esquerda):
            if flag:
                regiao_3[profundidade][1] = max(regiao_1[profundidade][0], esq)
            else:
                regiao_3 = None

        else:
            r_direita = [esq, np.PINF]
            #if interval_intersect(regiao_1[profundidade], r_direita):
            r1 = regiao_1[profundidade]
            r2 = r_direita
            min1, max1 = r1[0], r1[1]
            min2, max2 = r2[0], r2[1]
            flag = False if (max2 <= min1 or min2 >= max1) else True

            if flag:
                regiao_3[profundidade][0] = min(regiao_1[profundidade][1], esq)
            else:
                regiao_3 = None
        return regiao_3

    
    def procura(self, pontos_na_regiao, regiao, x, n):
        if (self.direita is None and self.esquerda is None):
             pontos_na_regiao.append([self.dado[-1], np.linalg.norm(x - self.dado[:-1])])
        else:
            regiao_esquerda = self.__intersecao_das_regioes(regiao, self.dado, 0)
            regiao_direita  = self.__intersecao_das_regioes(regiao, self.dado, 1)
            
            if regiao_esquerda is not None:
                # Calcula se r1 esta completamente contida dentro de r2
                flag = True
                r1 = regiao_esquerda
                r2 = regiao
                for i in range(len(r1)):
                    min1, max1 = r1[i][0], r1[i][1]    
                    min2, max2 = r2[i][0], r2[i][1]
                    if (max1 <= max2 and min1 >= min2) == 0:
                        flag = False
                        break

                if flag:
                    pontos_d = [[point[-1], np.linalg.norm(x - point[:-1])] for point in self.esquerda.percorre()]
                    pontos_na_regiao.extend(pontos_d)
                    pontos_na_regiao.sort(key = lambda point: point[1])
                    
                    if len(pontos_na_regiao) > n:
                        del pontos_na_regiao[n : -1]
                        
                else:
                    # Calcula a intersecao de duas regioes
                    r1 = regiao_esquerda
                    r2 = regiao
                    Flag = True
                    for i in range(len(r1)):
                        min1,max1 = r1[i][0],r1[i][1]    
                        min2,max2 = r2[i][0],r2[i][1]
                        if max2 <= min1 or min2 >= max1:
                            Flag = False
                            break

                    if flag:
                        self.esquerda.procura(pontos_na_regiao, regiao, x, n)
                    
            if regiao_direita is not None:
                # Calcula se r1 esta completamente contida dentro de r2
                flag = True
                r1 = regiao_direita
                r2 = regiao
                for i in range(len(r1)):
                    min1, max1 = r1[i][0], r1[i][1]    
                    min2, max2 = r2[i][0], r2[i][1]
                    if (max1 <= max2 and min1 >= min2) == 0:
                        flag = False
                        break

                if flag:
                    pontos_d = [[point[-1], np.linalg.norm(x - point[:-1])] for point in self.direita.percorre()]
                    pontos_na_regiao.extend(pontos_d)
                    pontos_na_regiao.sort(key = lambda point: point[1])
                    
                    if len(pontos_na_regiao) > n:
                        del pontos_na_regiao[n:-1]
                        
                else:
                # Calcula se r1 esta completamente contida dentro de r2
                    r1 = regiao_direita
                    r2 = regiao
                    Flag = True
                    for i in range(len(r1)):
                        min1,max1 = r1[i][0],r1[i][1]    
                        min2,max2 = r2[i][0],r2[i][1]
                        if max2 <= min1 or min2 >= max1:
                            Flag = False
                            break

                    if flag:
                        self.direita.procura(pontos_na_regiao, regiao, x, n)


    # distancia do ponto de referencia para construir a regiao inicial de busca
    def distancia_para_referencia(self, x):
        if (self.direita is None and self.esquerda is None):
            return np.linalg.norm(self.dado[:-1] - x)
        else:
            profundidade = self.dado[1]
            esq = self.dado[0]
            
            if self.esquerda and x[profundidade] <= esq:
                return self.esquerda.distancia_para_referencia(x)
                
            if self.direita and x[profundidade] > esq:
                return self.direita.distancia_para_referencia(x)


    def proximidade(self, x, n):
        pontos_na_regiao = []
        dim = self.distancia_para_referencia(x)/2.0
        incremento = dim
        while len(pontos_na_regiao) < n:
            pontos_na_regiao = []
            regiao = [[0, 0]] * len(x)
            for i in range(len(x)):
                regiao[i][0] = x[i] - dim
                regiao[i][1] = x[i] + dim

            self.procura(pontos_na_regiao, regiao, x, n)
            dim += incremento
        return pontos_na_regiao[:n]


### Implementação do xKNN:



In [12]:
class xKNN:
    def __init__(self):
        self.tree  = None
        self.max_n = None

        
    def treinamento(self, xTreinamento, yTreinamento):
        conj_de_treino = np.c_[xTreinamento, yTreinamento] 
        self.max_n = len(conj_de_treino)
        k = len(xTreinamento[0])
        self.tree = KdTree(k)
        self.tree.constroi(conj_de_treino)

    
    def __predicao(self, x, n):
        closest = self.tree.proximidade(x, n)
        outros = None
        conta = 0
        conta_elementos = closest[0][0]
        
        for elem in closest:
            if elem[0] == conta_elementos:
                conta += 1
            else:
                outros = elem[0]
        
        if conta > len(closest)/2:
            return conta_elementos
        elif conta == len(closest)/2:
            if np.random.randint(2):
                return conta_elementos
            else:
                return outros
        else:
            return outros

    
    def predicao(self, xTeste, n):
        predicao = []
        for point in xTeste:
            predicao.append(self.__predicao(point, n))
        return predicao

    
    def calcula_metricas(self, predicao, yTeste):
        tp = 0; fp = 0; tn = 0; fn = 0
        for pares in zip(predicao, yTeste):
            if pares == (0,0):    tn += 1
            elif pares == (0,1):  fn += 1
            elif pares == (1,0):  fp += 1
            elif pares == (1,1):  tp += 1
        acuracia  = 0 if len(predicao) == 0           else (tp + tn) / len(predicao)
        precisao  = 0 if (tp + fp) == 0               else tp / (tp + fp)
        revocacao = 0 if (tp + fn) == 0               else tp / (tp + fn)
        f1 = 0        if (precisao + revocacao) == 0  else (2 * revocacao * precisao) / (precisao + revocacao)
        return acuracia, precisao, revocacao, f1


### Execução e obtenção das métricas:



In [13]:
for f in os.listdir(BASE_DE_DADOS):
    if f.endswith(EXTENSAO_DOS_ARQUIVOS):
        #if f == "appendicitis.csv":        
        if f == "titanic.csv": # possui muito poucos pontos!
            numero_de_vizinhos = 2
        else:
            numero_de_vizinhos = NUM_DE_VIZINHOS
        
        pontos = pd.read_csv(BASE_DE_DADOS + f)
        dado_columns = pontos.columns[:-1]
        class_column = pontos.columns[-1] 

        x = pontos[dado_columns].to_numpy(dtype=np.float64)
        y = pontos[class_column].to_numpy()

        np.random.RandomState(SEMENTE_DE_ALEATORIDADE)
        mascara = np.random.rand(len(x)) < PORCAO_DE_TESTES
        xTreinamento = x[mascara]; xTeste = x[~mascara]
        yTreinamento = y[mascara]; yTeste = y[~mascara]

        medias = []; desvios_padrao = []
        for v in xTreinamento.T:
            medias.append(np.mean(v))
            desvios_padrao.append(np.std(v))
        
        for i in range(len(xTeste)):
            for j in range(len(xTeste[i])):
                if desvios_padrao[j] != 0:
                    xTeste[i][j] = (xTeste[i][j] - medias[j])/desvios_padrao[j]
                else:
                    xTeste[i][j] = xTeste[i][j] - medias[j]

        x_knn = xKNN()
        x_knn.treinamento(xTreinamento, yTreinamento)
        predicao = x_knn.predicao(xTeste, numero_de_vizinhos)

        acuracia, precisao, revocacao, metrica_f1 = x_knn.calcula_metricas(predicao, yTeste)
        print(f"nome do arquivo: \t{f}\nNum. tot. de ptos: \t{len(pontos)}\n"
              f"acuracia: \t\t{round(acuracia, 3)}\nprecisao: \t\t{round(precisao, 3)}\n"
              f"revocacao: \t\t{round(revocacao, 3)}\nmetrica F1: \t\t{round(metrica_f1, 3)}\n")


nome do arquivo: 	appendicitis.csv
Num. tot. de ptos: 	106
acuracia: 		0.593
precisao: 		0.267
revocacao: 		1.0
metrica F1: 		0.421

nome do arquivo: 	banana.csv
Num. tot. de ptos: 	5291
acuracia: 		0.792
precisao: 		0.767
revocacao: 		0.765
metrica F1: 		0.766

nome do arquivo: 	bupa.csv
Num. tot. de ptos: 	341
acuracia: 		0.59
precisao: 		0.5
revocacao: 		0.024
metrica F1: 		0.047

nome do arquivo: 	haberman.csv
Num. tot. de ptos: 	283
acuracia: 		0.742
precisao: 		0
revocacao: 		0.0
metrica F1: 		0

nome do arquivo: 	magic.csv
Num. tot. de ptos: 	18905
acuracia: 		0.647
precisao: 		0.647
revocacao: 		1.0
metrica F1: 		0.786

nome do arquivo: 	monk-2.csv
Num. tot. de ptos: 	432
acuracia: 		0.603
precisao: 		0.603
revocacao: 		1.0
metrica F1: 		0.752

nome do arquivo: 	phoneme.csv
Num. tot. de ptos: 	5349
acuracia: 		0.67
precisao: 		0.423
revocacao: 		0.421
metrica F1: 		0.422

nome do arquivo: 	pima.csv
Num. tot. de ptos: 	768
acuracia: 		0.682
precisao: 		0
revocacao: 		0.0
metrica

### Conclusão

O trabalho proporcionou um maior entendimento de um dos algoritmos geométricos visto em sala de aula, possibilitando abordagem dos aspectos práticos das árvores k-dimensionais para solução de problemas em outras áreas da computação.

### Referências

1. Computational Geometry algorithms and Applications. Berg et al. [Cap. 05]
2. Algorithm Design and Applications. Goodrich & Tamassia. [Cap. 21]
3. Aulas da disciplina: Algoritmos II; 2021-2; Prof.: Renato Vimieiro